In [2]:
import pandas as pd
import numpy as np
from io import StringIO


Imagina que eres un investigador en el mundo de Pokémon y has recibido una base de datos con información desactualizada y desordenada sobre avistamientos de Pokémon. Tu misión es limpiar y actualizar esta base de datos para que pueda ser utilizada en un estudio sobre la población de Pokémon en la región.

Datos Iniciales

Los datos iniciales contienen las siguientes columnas:

    SightingDate: Fecha del avistamiento.
    TrainerID: Identificación del entrenador que reportó el avistamiento.
    PokemonName: Nombre del Pokémon avistado.
    CP: Puntos de combate del Pokémon reportado.
    HP: Puntos de salud del Pokémon reportado.
    Type: Tipo del Pokémon.
    Weather: Clima durante el avistamiento.

# 1 Carga de Datos

In [29]:
# path_datos = 'pokemon.csv'
# TODO completa el codigo para cargar los datos
# df = pd.read_csv(path_datos)

# Especifica la ruta al archivo CSV y la codificación ISO
path_datos = 'pokemon.csv'
codificacion_iso = 'iso-8859-1'  # Asegúrate de usar la codificación correcta

# Carga los datos desde el archivo CSV
df = pd.read_csv(path_datos, encoding=codificacion_iso)
print(df)

               SightingDate    SightingTimeUTC TrainerID PokémonName       CP  \
0  2023-11-08T14:00:00+0000  2023-11-08T14:00Z     TR123     Pikachu      500   
1  2023-07-12T09:30:00+0100  2023-07-12T08:30Z     TR456       Eevee  MISSING   
2  2023/02/23T22:15:00+0900  2023-02-23T13:15Z     TR789    Magicarp    1,000   
3  2023-04-30T06:45:00-0400  2023-04-30T10:45Z     TR101      Gengar      800   
4  2023-08-15T16:00:00+1000                NaN     TR102   Bulbasaur      750   

   HP          Type        Weather  
0  35      Electric          Clear  
1  55        Normal         Cloudy  
2  10         Water           Rain  
3  45         Ghost  PARTLY_CLOUDY  
4  50  Grass/Poison          Sunny  


# Limpieza de Datos

## 1 Normalizacion de Zonas Horarias

Normaliza la columna `'SightingTimeUTC'` a la zona horaria UTC y convierte `'SightingDate'` al mismo formato de tiempo.


In [4]:
import pytz
#Investiga el formato al que pertence

df['SightingTimeUTC'].head()
print(df['SightingTimeUTC'])
#Por las entradas de la tabla sabemos que es UTF

print("Antes")
print(df['SightingDate'])

#Convierto SightingDate a UTF
#df['SightingDate'] = pd.to_datetime(df['SightingDate'], utc=True)
df['SightingDate'] = pd.to_datetime(df['SightingDate'], format='ISO8601')
print("Después")
print(df['SightingDate'])
  





0    2023-11-08T14:00Z
1    2023-07-12T08:30Z
2    2023-02-23T13:15Z
3    2023-04-30T10:45Z
4                  NaN
Name: SightingTimeUTC, dtype: object
Antes
0    2023-11-08T14:00:00+0000
1    2023-07-12T09:30:00+0100
2    2023/02/23T22:15:00+0900
3    2023-04-30T06:45:00-0400
4    2023-08-15T16:00:00+1000
Name: SightingDate, dtype: object
Después
0    2023-11-08 14:00:00+00:00
1    2023-07-12 09:30:00+01:00
2    2023-02-23 22:15:00+09:00
3    2023-04-30 06:45:00-04:00
4    2023-08-15 16:00:00+10:00
Name: SightingDate, dtype: object


Compara si la fecha de la columna `'SightingDate'` coincide con la fecha en `'SightingTimeUTC'` una vez normalizada.


In [5]:
df['SightingDate'] = pd.to_datetime(df['SightingDate'], utc=True)
df['SightingTimeUTC'] = pd.to_datetime(df['SightingTimeUTC'], utc=True)

# Compara si las fechas coinciden
df['Coincide'] = df['SightingDate'].dt.floor('T') == df['SightingTimeUTC'].dt.floor('T')

# Muestra el DataFrame resultante
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  Coincide  
0      500  35      Electric          Clear      True  
1  MISSING  55        Normal         Cloudy      True  
2    1,000  10         Water           Rain      True  
3      800  45         Ghost  PARTLY_CLOUDY      True  
4      750  50  Grass/Poison          Sunny     False  


Ajusta `'SightingTimeUTC'` a la zona horaria local de cada entrenador y crea una columna `'SightingTimeLocal'`.

In [6]:
#Mapeo de zonas horarias
zona_horaria_por_entrenador = {
    'TR123': 'America/Mexico_City',
    'TR456': 'Europe/London',
    'TR789': 'Asia/Tokyo',
    'TR101': 'America/New_York',
    'TR102': 'Australia/Sydney',
    # Agrega más mapeos según sea necesario
}

# Agrega una nueva columna 'ZonaHoraria' al DataFrame, utilizando una zona horaria predeterminada para NaN
df['ZonaHoraria'] = df['TrainerID'].map(zona_horaria_por_entrenador).fillna('UTC')

# Ajusta la zona horaria de la columna 'SightingTimeUTC' utilizando la nueva columna 'ZonaHoraria'
df['SightingTimeLocal'] = df.apply(lambda row: row['SightingTimeUTC'].astimezone(pytz.timezone(row['ZonaHoraria'])) if pd.notnull(row['SightingTimeUTC']) else pd.NaT, axis=1)

# Imprime el DataFrame resultante
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  Coincide          ZonaHoraria  \
0      500  35      Electric          Clear      True  America/Mexico_City   
1  MISSING  55        Normal         Cloudy      True        Europe/London   
2    1,000  10         Water           Rain      True           Asia/Tokyo   
3      800  45         Ghost  PARTLY_CLOUDY      True     America/New_York   
4      750  50  Grass/Poison          Sunny     False     Australia/Sydney   

           SightingTimeLocal  
0  2023-11-08 08:00:00-06

Calcula el tiempo transcurrido desde el momento del avistamiento hasta `'ahora'` (tu hora local) y crea una columna `'TimeSinceSighting'`.

In [13]:
import datetime

"""
# Obtén la hora actual en tu zona horaria local
hora_actual_local = datetime.datetime.now(pytz.timezone('America/New_York'))

# Convierte la columna 'SightingTimeLocal' a datetime64 con información de zona horaria UTC
df['SightingTimeLocal'] = df['SightingTimeLocal'].dt.tz_convert('UTC')

# Calcula el tiempo transcurrido desde el avistamiento hasta ahora
df['TimeSinceSighting'] = (hora_actual_local - df['SightingTimeLocal']).dt.total_seconds() / 60  # En minutos
"""
# Obtén la hora actual en tu zona horaria local
hora_actual_local = datetime.datetime.now(pytz.timezone('America/New_York'))
# Asegúrate de que 'SightingTimeLocal' está en formato datetime y tiene zonas horarias asignadas
df['SightingTimeLocal'] = pd.to_datetime(df['SightingTimeLocal'], utc=True)

# Calcula el tiempo transcurrido desde el avistamiento hasta ahora
df['TimeSinceSighting'] = (hora_actual_local - df['SightingTimeLocal']).dt.total_seconds() / 60  # En minutos

# Imprime el DataFrame resultante
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  Coincide          ZonaHoraria  \
0      500  35      Electric          Clear      True  America/Mexico_City   
1  MISSING  55        Normal         Cloudy      True        Europe/London   
2    1,000  10         Water           Rain      True           Asia/Tokyo   
3      800  45         Ghost  PARTLY_CLOUDY      True     America/New_York   
4      750  50  Grass/Poison          Sunny     False     Australia/Sydney   

          SightingTimeLocal  TimeSinceSighting  
0 2023-

"\n# Obtén la hora actual en tu zona horaria local\nhora_actual_local = datetime.datetime.now(pytz.timezone('tu_zona_horaria_local'))\n\n# Convierte la columna 'SightingTimeLocal' a objeto datetime si aún no lo está\ndf['SightingTimeLocal'] = pd.to_datetime(df['SightingTimeLocal'])\n\n# Asegúrate de que ambos objetos de fecha y hora tengan la misma zona horaria\ndf['SightingTimeLocal'] = df['SightingTimeLocal'].dt.tz_localize(pytz.timezone('tu_zona_horaria_local'))\n\n# Calcula el tiempo transcurrido desde el avistamiento hasta ahora\ndf['TimeSinceSighting'] = (hora_actual_local - df['SightingTimeLocal']).dt.total_seconds() / 60  # En minutos\n"

## 2. Limpeiza de IDs

Llena los valores faltantes en `'TrainerID'` con el ID `'UNKNOWN'`.


In [14]:
df['TrainerID'].fillna('UNKNOWN', inplace=True)
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  Coincide          ZonaHoraria  \
0      500  35      Electric          Clear      True  America/Mexico_City   
1  MISSING  55        Normal         Cloudy      True        Europe/London   
2    1,000  10         Water           Rain      True           Asia/Tokyo   
3      800  45         Ghost  PARTLY_CLOUDY      True     America/New_York   
4      750  50  Grass/Poison          Sunny     False     Australia/Sydney   

          SightingTimeLocal  TimeSinceSighting  
0 2023-

## 3. Corrección de Nombres de Pokémon

Asegúrate de que los nombres de Pokémon estén capitalizados correctamente.


In [15]:
df['PokemonName'] =  df['PokémonName'].str.capitalize()

# Imprime el DataFrame resultante
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  Coincide          ZonaHoraria  \
0      500  35      Electric          Clear      True  America/Mexico_City   
1  MISSING  55        Normal         Cloudy      True        Europe/London   
2    1,000  10         Water           Rain      True           Asia/Tokyo   
3      800  45         Ghost  PARTLY_CLOUDY      True     America/New_York   
4      750  50  Grass/Poison          Sunny     False     Australia/Sydney   

          SightingTimeLocal  TimeSinceSighting PokemonNa

## 4. Conversión de 'CP' y 'HP' a Numéricos

Convierte `'CP'` y `'HP'` a valores numéricos, manejando los `'MISSING'` y comas como separadores de miles.

In [20]:


# Elimina las comas y convierte a valores numéricos
df['CP'] = pd.to_numeric(df['CP'].astype(str).str.replace(',', ''), errors='coerce')
df['HP'] = pd.to_numeric(df['HP'].astype(str).str.replace(',', ''), errors='coerce')

df['CP'] = df['CP'].replace('MISSING', pd.NA)
df['HP'] = df['HP'].replace('MISSING', pd.NA)

# Imprime el DataFrame resultante
print(df)

               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

       CP  HP          Type        Weather  Coincide          ZonaHoraria  \
0   500.0  35      Electric          Clear      True  America/Mexico_City   
1     NaN  55        Normal         Cloudy      True        Europe/London   
2  1000.0  10         Water           Rain      True           Asia/Tokyo   
3   800.0  45         Ghost  PARTLY_CLOUDY      True     America/New_York   
4   750.0  50  Grass/Poison          Sunny     False     Australia/Sydney   

          SightingTimeLocal  TimeSinceSighting PokemonName  
0

## 5. Estandarización de 'Type'

Divide la columna `'Type'` en `'PrimaryType'` y `'SecondaryType'` cuando hay dos tipos.


In [25]:
# Divide la columna 'Type' en dos columnas: 'PrimaryType' y 'SecondaryType'

df[['PrimaryType', 'SecondaryType']] = df['Type'].str.split('/', n=1, expand=True)



# Imprime el DataFrame resultante
print(df)

               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

       CP  HP          Type        Weather  Coincide          ZonaHoraria  \
0   500.0  35      Electric          Clear      True  America/Mexico_City   
1     NaN  55        Normal         Cloudy      True        Europe/London   
2  1000.0  10         Water           Rain      True           Asia/Tokyo   
3   800.0  45         Ghost  PARTLY_CLOUDY      True     America/New_York   
4   750.0  50  Grass/Poison          Sunny     False     Australia/Sydney   

          SightingTimeLocal  TimeSinceSighting PokemonName Pri

## 6. Corrección del Clima

Estándariza la columna `'Weather'` para que todos los valores sean mayúsculas.


In [30]:
df['Weather'] = df['Weather'].str.upper()
# Imprime el DataFrame actualizado
print(df)


               SightingDate    SightingTimeUTC TrainerID PokémonName       CP  \
0  2023-11-08T14:00:00+0000  2023-11-08T14:00Z     TR123     Pikachu      500   
1  2023-07-12T09:30:00+0100  2023-07-12T08:30Z     TR456       Eevee  MISSING   
2  2023/02/23T22:15:00+0900  2023-02-23T13:15Z     TR789    Magicarp    1,000   
3  2023-04-30T06:45:00-0400  2023-04-30T10:45Z     TR101      Gengar      800   
4  2023-08-15T16:00:00+1000                NaN     TR102   Bulbasaur      750   

   HP          Type        Weather  
0  35      Electric          CLEAR  
1  55        Normal         CLOUDY  
2  10         Water           RAIN  
3  45         Ghost  PARTLY_CLOUDY  
4  50  Grass/Poison          SUNNY  


# 3 Analisis de Datos

## Agrupaciones

Agrupa el DataFrame por `'Type'` y calcula la suma de `'CP'` para cada grupo.

In [31]:
# Agrupa por 'Type' y suma 'CP' para cada grupo
sum_cp_by_type = df.groupby('Type')['CP'].sum()

# Imprime el resultado
print(sum_cp_by_type)

Type
Electric            500
Ghost               800
Grass/Poison        750
Normal          MISSING
Water             1,000
Name: CP, dtype: object


Después de la suma, agrega una columna que calcule la media de `'HP'` por cada `'Type'`, pero solo para aquellos Pokémon cuyo `'CP'` sea mayor que el promedio de `'CP'` de todo el DataFrame.

In [33]:
# Convierte la columna 'CP' a numérica, ignorando los valores 'MISSING'
df['CP'] = pd.to_numeric(df['CP'], errors='coerce')

# Calcula el promedio de 'CP' de todo el DataFrame
promedio_cp_total = df['CP'].mean()

# Filtra los Pokémon cuyo 'CP' sea mayor que el promedio total
pokemon_cp_mayor_promedio = df[df['CP'] > promedio_cp_total]

# Agrupa por 'Type' y calcula la suma de 'CP' y la media de 'HP' para esos Pokémon
resultados_agregados = pokemon_cp_mayor_promedio.groupby('Type').agg({'CP': 'sum', 'HP': 'mean'}).reset_index()

# Renombra las columnas resultantes
resultados_agregados.columns = ['Type', 'Suma_CP', 'Media_HP']

# Fusiona los resultados agregados de nuevo con el DataFrame original
df = pd.merge(df, resultados_agregados, on='Type', how='left')

# Imprime el DataFrame resultante
print(df)

               SightingDate    SightingTimeUTC TrainerID PokémonName     CP  \
0  2023-11-08T14:00:00+0000  2023-11-08T14:00Z     TR123     Pikachu  500.0   
1  2023-07-12T09:30:00+0100  2023-07-12T08:30Z     TR456       Eevee    NaN   
2  2023/02/23T22:15:00+0900  2023-02-23T13:15Z     TR789    Magicarp    NaN   
3  2023-04-30T06:45:00-0400  2023-04-30T10:45Z     TR101      Gengar  800.0   
4  2023-08-15T16:00:00+1000                NaN     TR102   Bulbasaur  750.0   

   HP          Type        Weather  Suma_CP  Media_HP  
0  35      Electric          CLEAR      NaN       NaN  
1  55        Normal         CLOUDY      NaN       NaN  
2  10         Water           RAIN      NaN       NaN  
3  45         Ghost  PARTLY_CLOUDY    800.0      45.0  
4  50  Grass/Poison          SUNNY    750.0      50.0  
